In [1]:

using Revise
includet("../general_modules/channels.jl")
include("../general_modules/mesh.jl")
using .channels
using .mesh
using JSON

In [2]:
data = JSON.parsefile("test.json")
Systems_params= data["Systems"]
mesh_params= data["Mesh"]
fermion=false
α= α3b(fermion,0.0,0.0,1,Systems_params["lmax"],Systems_params["lmin"],Systems_params["λmax"],Systems_params["λmin"],Systems_params["s1"],Systems_params["s2"],Systems_params["s3"],Systems_params["t1"],Systems_params["t2"],Systems_params["t3"],Systems_params["MT"])

grid= initialmesh(mesh_params["nθ"],mesh_params["nx"],mesh_params["ny"],Float64(mesh_params["xmax"]),Float64(mesh_params["ymax"]),Float64(mesh_params["alpha"]))

For J=0.0 T=0.0 parity=1 Number of channels: 1
scaling factor for x: 0.3


Main.mesh.meshset(12, 12, 12, [-0.9815606342467192, -0.9041172563704748, -0.7699026741943047, -0.5873179542866175, -0.3678314989981802, -0.1252334085114689, 0.1252334085114689, 0.3678314989981802, 0.5873179542866175, 0.7699026741943047, 0.9041172563704748, 0.9815606342467192], [0.04717533638651175, 0.10693932599531826, 0.16007832854334625, 0.20316742672306587, 0.23349253653835492, 0.24914704581340288, 0.24914704581340288, 0.23349253653835492, 0.20316742672306587, 0.16007832854334625, 0.10693932599531826, 0.04717533638651175], [0.0848575044719742, 0.2856978124093846, 0.6049476415733046, 1.0477062209334076, 1.6216473060047178, 2.337844182123722, 3.2122166066967295, 4.268145709136995, 5.5415989902690015, 7.092535125720268, 9.036017587831902, 11.666785312828601], [0.14277897009800458, 0.259383321803658, 0.37995967777078926, 0.5068231073270444, 0.6428816610948332, 0.7921245262743847, 0.960438937417142, 1.1572465340299454, 1.3992307171257872, 1.7204236864089044, 2.2079451522012565, 3.2157606

In [ ]:
for i in 1:mesh_params["nx"]
    println(grid.xi[i], "   ", grid.dxi[i])
end

In [ ]:
using Revise
Revise.revise()
includet("Gcoefficient.jl")
using .Gcoefficient

initialY(λmax, lmax, nθ, nx, ny, cosθi, xi, yi, P::Char)
return Yλαout, Ylαin, Yλαin

Yλin[iθ, iy, ix, nch, perm_index]

In [ ]:
Gαα=computeGcoefficient(α, grid)

In [ ]:
using Revise
includet("twobody.jl")
using .twobodybound

In [ ]:
potname="MT"
bound_states=bound2b(grid, potname)

In [ ]:
using Revise
includet("spline.jl")
using .spline

In [ ]:
"""
Test the cubic Hermitian spline interpolation with a polynomial function.
"""
# Import Printf for formatting
using Printf
using Plots
function test_cubherm_interpolation()
    # Define the polynomial function f(x) = 5x^5 + 4x^4 + 3x^3 + 2x^2 + x
    f(x) = 5*x^5 + 4*x^4 + 3*x^3 + 2*x^2 + x
    
    # Create a set of old grid points (coarse grid)
    xold = collect(range(-1.0, 1.0, length=15))
    
    # Evaluate the function at the old grid points
    yold = f.(xold)
    
    # println("Original grid points:")
    # for i in 1:length(xold)
    #     println("x = $(xold[i]), f(x) = $(yold[i])")
    # end
    # println()
    
    # Create a set of test points (fine grid) where we want to interpolate
    xtest = collect(range(-0.95, 0.95, length=20))
    
    # Compute the exact values
    yexact = f.(xtest)
    
    # Interpolate using our cubic Hermitian spline
    yinterp = Float64[]
    for x in xtest
        push!(yinterp, cubherm_interp_point(xold, yold, x))
    end
    
    # Compute errors
    errors = abs.(yinterp - yexact)
    max_error = maximum(errors)
    avg_error = sum(errors) / length(errors)
    
    println("Interpolation results and errors:")
    println("   x       Exact        Interpolated       Error")
    println("------------------------------------------------------")
    for i in 1:length(xtest)
        # Use simple string formatting instead of @printf/@sprintf
        x_str = @sprintf("%.4f", xtest[i])
        exact_str = @sprintf("%.6f", yexact[i])
        interp_str = @sprintf("%.6f", yinterp[i])
        error_str = @sprintf("%.6e", errors[i])
        
        println("$(lpad(x_str, 7))  $(lpad(exact_str, 12))  $(lpad(interp_str, 12))  $(lpad(error_str, 12))")
    end
    
    println("\nMaximum error: $max_error")
    println("Average error: $avg_error")
    
    # Optional: Plot the results if Plots.jl is available
    try
        
        
        p = plot(xold, yold, seriestype=:scatter, 
             label="Original Points", markersize=6, legend=:topleft)
        
        plot!(p, xtest, yexact, 
              label="Exact Function", linewidth=2)
        
        plot!(p, xtest, yinterp, 
              label="Interpolated", linewidth=2, linestyle=:dash)
        
        # Plot error
        p_error = plot(xtest, errors, 
                   label="Interpolation Error", linewidth=2,
                   xlabel="x", ylabel="Error", title="Interpolation Error")
        
        # Display both plots
        plot(p, p_error, layout=(2,1), size=(800, 600),
             title=["Cubic Hermitian Spline Interpolation" ""])
        
        savefig("interpolation_test.png")
        println("\nPlot saved as 'interpolation_test.png'")
    catch
        println("\nPlotting skipped (Plots.jl not available)")
    end
    
    return max_error, avg_error
end

In [ ]:
using Printf 
test_cubherm_interpolation()

In [3]:
using Revise
includet("matrices.jl")
using .matrices


In [5]:
Rxy_matrix(α, grid)


πb=0.06178521741456235 - 0.0im    ξb=-2.4031338031971625 + 0.0im adj_factor=-0.010556680543496608 - 0.0im
πb=0.011792154258108539 - 0.0im    ξb=-2.610014465270078 + 0.0im adj_factor=-0.02109176186923329 - 0.0im
πb=-0.07827508573244231 + 0.0im    ξb=-2.8072479816230973 + 0.0im adj_factor=-0.027029548902614272 - 0.0im
πb=-0.2079146358629098 + 0.0im    ξb=-2.883224948585327 + 0.0im adj_factor=-0.0297472496667578 - 0.0im
πb=-0.3748479621288843 + 0.0im    ξb=-2.8154094205875655 + 0.0im adj_factor=-0.030457710313923095 - 0.0im
πb=-0.5737027936980423 + 0.0im    ξb=-2.6350946667894624 + 0.0im adj_factor=-0.029800223888430448 - 0.0im
πb=-0.7948833871492997 + 0.0im    ξb=-2.391248998272984 + 0.0im adj_factor=-0.02805093518494287 - 0.0im
πb=-1.024195507273595 + 0.0im    ξb=-2.1300524802978753 + 0.0im adj_factor=-0.02529993069416231 - 0.0im
πb=-1.2437478454497273 + 0.0im    ξb=-1.8868780302675925 + 0.0im adj_factor=-0.021559168163726376 - 0.0im
πb=-1.4342625140372414 + 0.0im    ξb=-1.6853311100553

144×144 Matrix{ComplexF64}:
    1.94852+0.0im    0.0034192+0.0im  …   -5.15856e-8+0.0im
  -0.337873+0.0im     0.174072+0.0im       -2.5023e-7+0.0im
  0.0381485+0.0im     0.188747+0.0im       2.61821e-7+0.0im
  0.0402584+0.0im    0.0422297+0.0im       8.61574e-7+0.0im
 -0.0420445+0.0im   -0.0316838+0.0im      -2.63837e-6+0.0im
  0.0191563+0.0im    0.0127837+0.0im  …     2.9208e-6+0.0im
 0.00151422+0.0im  0.000962109+0.0im       4.77159e-7+0.0im
  -0.010878+0.0im   -0.0065754+0.0im      -7.68847e-6+0.0im
  0.0110424+0.0im   0.00650472+0.0im       1.66449e-5+0.0im
 -0.0075271+0.0im  -0.00435475+0.0im      -2.56051e-5+0.0im
           ⋮                          ⋱  
  0.0406825+0.0im    0.0226536+0.0im  …    0.00173539+0.0im
 -0.0247169+0.0im   -0.0138007+0.0im     -0.000647711+0.0im
  0.0413291+0.0im    0.0230143+0.0im       0.00181754+0.0im
 -0.0623988+0.0im   -0.0347272+0.0im      -0.00455561+0.0im
 -0.0159169+0.0im  -0.00900873+0.0im      -0.00184246+0.0im
   0.124775+0.0im    0.0702547

In [3]:
using Revise
includet("threebodybound.jl")
using .threebodybound

In [5]:
ThreeBody_Bound(α, grid)

Eigenvalues: ComplexF64[-1.3959983610585638 - 9.72959394763678e-16im, -1.3623740423580009 + 0.1247560997516654im, -1.362374042358 - 0.12475609975166423im, -1.2111127049652333 + 8.286768017225527e-18im, -1.1110155905408727 + 1.6680796792113412e-16im, -0.9860373407379334 + 1.7555655222849478e-16im, -0.8971646194485373 + 7.545357088099484e-17im, -0.8332558217402489 + 1.4542197828432386e-16im, -0.7626719745508997 - 6.434578658918425e-16im, -0.7128865972629959 + 8.853429116651531e-16im, -0.5624069127173577 + 7.844527272051396e-17im, -0.39752969268514476 - 4.923323578628199e-16im, -0.2961224768805773 + 1.97758476261356e-16im, -0.28208086762892015 + 5.504250307388321e-17im, -0.2710780653632811 - 0.3222664853388777im, -0.27107806536328083 + 0.3222664853388777im, -0.25148873235447444 + 0.11659283289935174im, -0.2514887323544739 - 0.11659283289935178im, -0.22885044684784053 + 2.1007860753828417e-16im, -0.22067441496913992 + 0.06867543702870829im, -0.22067441496913934 - 0.06867543702870803im, -0.

12-element Vector{Any}:
 (-1.3959983610585638 - 9.72959394763678e-16im, ComplexF64[4.66664155719818e-6 - 1.6239950939469887e-17im, -0.00106623031737824 + 5.366800753803247e-18im, 0.003516323469427149 + 1.6371452804531117e-17im, -0.007215014334264602 - 5.4643789493269423e-17im, 0.0016131756178241567 - 1.235990476633475e-17im, 0.011870547022047574 - 5.2475385148298415e-17im, 0.003629374954820203 + 6.21247844834194e-17im, -0.012617061558832198 + 4.119968255444917e-17im, 0.0030999656947748055 - 3.122502256758253e-17im, 0.0010582062411697846 - 2.8189256484623115e-18im  …  -0.00034458885456479295 + 4.2609145378680324e-17im, -0.0012581386011801536 + 7.94178091345632e-17im, -0.0038372403459223502 + 1.1080546202801855e-16im, -0.008789462668766047 + 1.5395670849294163e-16im, -0.014195311331028198 + 2.1597307275911248e-16im, -0.008742727135697358 + 1.448494102440634e-16im, 0.029730409095991603 - 8.673617379884035e-18im, 0.04313973975904918 - 1.7867651802561113e-16im, 0.013281617794728064 - 4.2891

In [ ]:
T_matrix(α,grid) 

In [ ]:
V_matrix(α, grid, "MT")